# Check the annotion of LC-MS identified metabolites 

We check the annotation quality (potential for being an isomere or contamination) for the metabolites used in the MFA and DIABLO. 

In [103]:
import pandas as pd
import numpy as np
import os
import re

In [104]:
# set the directory 
os.chdir('/home/lfloerl/cloud/lfloerl/Microterroir/LC-MS_data/Results')

In [105]:
file_path_MS2_neg = "/home/lfloerl/cloud/lfloerl/Microterroir/LC-MS_data/FGCZ data/p32837_o34707_Lena_RPMetabolomics_MS2_Features_NEG.xlsx"

# Get the raw data
file_path_MS2_pos = "/home/lfloerl/cloud/lfloerl/Microterroir/LC-MS_data/FGCZ data/p32837_o34707_Lena_RPMetabolomics_MS2_Features_POS.xlsx"

# Read only the 'Area_Stats' sheet
df_neg = pd.read_excel(file_path_MS2_neg, sheet_name="Compounds")
df_pos = pd.read_excel(file_path_MS2_pos, sheet_name="Compounds")

In [106]:
# for sure no contaminations & correctly annotated 

subset_pos = df_pos[
    (df_pos["mzCloud Best Match"] > 80) &
    (df_pos["Ratio: (PostMLF) / (Blank)"] > 3)
]
print(df_pos.shape)
print(subset_pos.shape)

(191, 377)
(22, 377)


In [107]:
subset_neg = df_neg[
    (df_neg["mzCloud Best Match"] > 80) &
    (df_neg["Ratio: (PostMLF) / (Blank)"] > 3)
]
print(df_neg.shape)
print(subset_neg.shape)

(170, 381)
(28, 381)


In [108]:
df_pos.head()

,Tags,Checked,Name,Formula,Annot. Source: Predicted Compositions,Annot. Source: mzCloud Search,Annot. Source: mzVault Search,Annot. Source: ChemSpider Search,Annot. Source: MassList Search,Annot. DeltaMass [Da],...,Peak Rating: 34707_43_PostMLF_2023_Plot4.raw (F91),Peak Rating: 34707_44_PostMLF_2023_Plot15.raw (F93),Peak Rating: 34707_45_PostMLF_2023_Plot5.raw (F95),Peak Rating: 34707_46_PostMLF_2023_Plot20.raw (F97),Peak Rating: 34707_47_PostMLF_2023_Plot17.raw (F99),Peak Rating: 34707_48_PostMLF_2023_Plot13.raw (F101),Peak Rating: 34707_49_PostMLF_2023_Plot14.raw (F103),Peak Rating: 34707_50_PostMLF_2023_Plot19.raw (F105),Peak Rating: 34707_51_PostMLF_2023_Plot9.raw (F107),Peak Rating: 34707_52_PostMLF_2023_Plot12.raw (F109)
0,NaN,False,"1,2-Di-(9Z,12Z,15Z-octadecatrienoyl)-sn-glycer...",C44 H76 N O8 P,Not the top hit,No results,Full match,Partial match,Partial match,-0.00113,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,True,"1,2-Dilinoleoyl-sn-glycero-3-phosphocholine",C44 H80 N O8 P,Full match,No results,Full match,Partial match,Partial match,-0.00102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A,True,"1,2-Dipalmitoleoyl-sn-glycero-3-phosphocholine",C40 H76 N O8 P,Not the top hit,No results,Full match,Full match,Not the top hit,-0.00112,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,False,"1,5-Naphthalenediamine",C10 H10 N2,No results,Full match,No results,Full match,No results,-0.00017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,False,"1,6-ANHYDRO-B-GLUCOSE",C6 H10 O5,Full match,No results,Full match,Partial match,Not the top hit,-0.00020,...,NaN,2.3,NaN,5.8,4.2,NaN,4.6,NaN,NaN,NaN


In [109]:
df_pos.columns

Index(['Tags', 'Checked', 'Name', 'Formula',
       'Annot. Source: Predicted Compositions',
       'Annot. Source: mzCloud Search', 'Annot. Source: mzVault Search',
       'Annot. Source: ChemSpider Search', 'Annot. Source: MassList Search',
       'Annot. DeltaMass [Da]',
       ...
       'Peak Rating: 34707_43_PostMLF_2023_Plot4.raw (F91)',
       'Peak Rating: 34707_44_PostMLF_2023_Plot15.raw (F93)',
       'Peak Rating: 34707_45_PostMLF_2023_Plot5.raw (F95)',
       'Peak Rating: 34707_46_PostMLF_2023_Plot20.raw (F97)',
       'Peak Rating: 34707_47_PostMLF_2023_Plot17.raw (F99)',
       'Peak Rating: 34707_48_PostMLF_2023_Plot13.raw (F101)',
       'Peak Rating: 34707_49_PostMLF_2023_Plot14.raw (F103)',
       'Peak Rating: 34707_50_PostMLF_2023_Plot19.raw (F105)',
       'Peak Rating: 34707_51_PostMLF_2023_Plot9.raw (F107)',
       'Peak Rating: 34707_52_PostMLF_2023_Plot12.raw (F109)'],
      dtype='object', length=377)

### Define the metabolites we are looking for 
--> from the DIABLO model

In [121]:
# this is the MFA table 
#MFA_table = pd.read_csv('/home/lfloerl/cloud/lfloerl/Microterroir/LC-MS_data/Results/MicrobiomeMetabolome/PostMLF_MicrobiomeMetabolome/MFA_table.csv')
MFA_table = pd.read_csv('/home/lfloerl/cloud/lfloerl/Microterroir/LC-MS_data/Results/MicrobiomeMetabolome/SensoryMicrobiomeMetabolome/MFA_table.csv')



MFA_table.head()

,Unnamed: 0,LCMS_Pos,LCMS_Pos.1,LCMS_Pos.2,LCMS_Pos.3,LCMS_Pos.4,LCMS_Pos.5,LCMS_Pos.6,LCMS_Pos.7,LCMS_Pos.8,...,Sensory.11,Sensory.12,Sensory.13,Sensory.14,Sensory.15,Sensory.16,Sensory.17,Sensory.18,Sensory.19,Sensory.20
0,NaN,"1,2-Di-(9Z,12Z,15Z-octadecatrienoyl)-sn-glycer...","1,2-Dilinoleoyl-sn-glycero-3-phosphocholine","1,2-Dipalmitoleoyl-sn-glycero-3-phosphocholine","1,5-Naphthalenediamine","1,6-ANHYDRO-B-GLUCOSE","1-(3,4-dimethoxyphenyl)ethan-1-one oxime",1-Palmitoyl-sn-glycero-3-phosphocholine,1-Tetradecylamine,"2,2,6,6-Tetramethyl-4-piperidinol",...,Olfactory_complexity,CO2,Acidity,Bitterness,Sweetness,Volume_in_the_mouth,Balance,Alcohol,Freshness_on_the_palate,Aromatic_length
1,0.0,-0.6534525941912974,-0.5088549337187033,-0.6218214660130147,-0.5756880168559823,-0.8565135440207675,0.7610378343036935,-0.6817774240868849,-0.8501547734799249,-0.8126931168058683,...,2.94,0.145,4.085,2.69,1.43,0.68,1.34,2.01,2.47,2.79
2,1.0,-0.4126319732390702,-0.5271657880077699,-0.5469337515057482,-0.3599543261597051,-0.5508048450895325,0.658860031568262,-0.301486505930976,0.1679071722619442,-0.7733613860014835,...,4.795,0.435,4.9,2.465,2.28,3.08,2.52,1.82,3.255,4.555
3,2.0,-0.5903673032530983,-0.5034352240991123,-0.5986697497697301,-0.4433376875243351,-0.6996237162904174,0.793922886490035,-0.4305276536784228,-0.6571974140870772,-0.1967866511062013,...,3.24,0.79,5.245,2.05,2.775,3.425,2.22,2.095,2.515,5.155
4,3.0,-0.5183640568488954,-0.3977652500023097,-0.4328277824825585,-0.5047011813584109,-1.4853738081510766,0.5435421837301497,-0.5066526028588746,-0.4021625020937598,-0.7241851103404461,...,2.955,0.06,4.34,1.995,0.88,1.89,2.3,1.32,3.355,3.635


In [122]:
metabolite_names = MFA_table.iloc[0].tolist()
# drop the sample name etc
metabolite_names = [m for m in metabolite_names if isinstance(m, str) and m not in ['Sample Name', 'plot', 'year']]

metabolite_names

['1,2-Di-(9Z,12Z,15Z-octadecatrienoyl)-sn-glycero-3-phosphocholine',
 '1,2-Dilinoleoyl-sn-glycero-3-phosphocholine',
 '1,2-Dipalmitoleoyl-sn-glycero-3-phosphocholine',
 '1,5-Naphthalenediamine',
 '1,6-ANHYDRO-B-GLUCOSE',
 '1-(3,4-dimethoxyphenyl)ethan-1-one oxime',
 '1-Palmitoyl-sn-glycero-3-phosphocholine',
 '1-Tetradecylamine',
 '2,2,6,6-Tetramethyl-4-piperidinol',
 '2,2-difluoro-N-[2-methyl-6-(trifluoromethyl)pyridin-3-yl]-2-(phenylthio)acetamide',
 '2-(2-amino-3-methylbutanamido)-3-phenylpropanoic acid',
 '2-[(4-chlorophenyl)sulfonyl]-3-[(2-furylmethyl)amino]-3-(methylthio)acrylonitrile',
 '2-[1-(4-isobutylphenyl)ethyl]-5-(3-nitrophenyl)-1,3,4-oxadiazole',
 '2-Amino-1,3,4-octadecanetriol',
 '2-Amino-1,3-octadecanediol',
 '3-(1-naphthylmethylidene)indolin-2-one',
 '3-(2-Hydroxyethyl)indole',
 '3-(tert-butyl)-N-[2-hydroxy-3-(4-methoxyphenoxy)propyl]-1-methyl-1H-pyrazole-5-carboxamide',
 '3-phenyl-5-(1,2,3-thiadiazol-4-yl)-1,2,4-oxadiazole',
 '4-Acetamidobutanoic acid',
 '4-Acetamidob

#### Look up metabolites 
are they from GC-MS or LC-MS? 

In [123]:
first_row = MFA_table.iloc[0]
# Create mapping: entry (from first row) -> column name
entry_to_column = {str(value).strip(): col for col, value in first_row.items()}

In [124]:
def find_columns_for_entry(entry, entry_dict):
    # Normalize search entry
    pattern = re.sub(r'\s+', r'\\s*', entry.strip())  # allow for any spacing
    pattern = re.escape(pattern).replace(r'\,', ',')  # escape but allow comma
    
    matches = []  # List to store all matching results
    
    for key, col in entry_dict.items():
        if re.search(pattern, key, flags=re.IGNORECASE):
            matches.append((key, col))
    
    return matches if matches else None  # Return matches or None if no match found


### Extract annotation qualtiy (all MFA)

In [125]:
# List of columns to extract
columns_to_extract = [
    'Name', 'Formula',
    'Annot. Source: Predicted Compositions',
    'Annot. Source: mzCloud Search', 'Annot. Source: mzVault Search',
    'Annot. Source: ChemSpider Search', 'Annot. Source: MassList Search',
    'Annot. DeltaMass [Da]', 'mzCloud Best Match', 'mzCloud Best Match Confidence', 'Ratio: (PostMLF) / (Blank)'
]

# Storage
results = []
not_found = []

def normalize(s):
    if not isinstance(s, str):
        return ""
    # Remove non-alphanumerics and trailing numeric suffixes like '.1', ' n6.2', etc.
    s = re.sub(r'(\.\d+|\sn\d+(\.\d+)?)$', '', s)
    return re.sub(r'[^a-zA-Z0-9]', '', s).lower()

# Search function using best match (substring after normalization)
def find_best_match(met_name, df):
    target = normalize(met_name)
    for idx, row in df.iterrows():
        candidate = normalize(str(row['Name']))
        if target in candidate:
            return row
    return None

# Check both dataframes
for met in metabolite_names:
    match = find_best_match(met, df_pos)
    if match is None:
        match = find_best_match(met, df_neg)

    if match is not None:
        results.append(match[columns_to_extract])
    else:
        not_found.append(met)

# Combine all results into a new dataframe
result_df = pd.DataFrame(results)


In [126]:
result_df.to_csv('LCMS-MS2-DIABLOmetabolites-annotation.csv')

In [127]:
result_df

,Name,Formula,Annot. Source: Predicted Compositions,Annot. Source: mzCloud Search,Annot. Source: mzVault Search,Annot. Source: ChemSpider Search,Annot. Source: MassList Search,Annot. DeltaMass [Da],mzCloud Best Match,mzCloud Best Match Confidence,Ratio: (PostMLF) / (Blank)
0,"1,2-Di-(9Z,12Z,15Z-octadecatrienoyl)-sn-glycer...",C44 H76 N O8 P,Not the top hit,No results,Full match,Partial match,Partial match,-0.00113,NaN,NaN,0.287
1,"1,2-Dilinoleoyl-sn-glycero-3-phosphocholine",C44 H80 N O8 P,Full match,No results,Full match,Partial match,Partial match,-0.00102,NaN,NaN,0.310
2,"1,2-Dipalmitoleoyl-sn-glycero-3-phosphocholine",C40 H76 N O8 P,Not the top hit,No results,Full match,Full match,Not the top hit,-0.00112,NaN,NaN,0.196
3,"1,5-Naphthalenediamine",C10 H10 N2,No results,Full match,No results,Full match,No results,-0.00017,78.6,8.9,0.351
4,"1,6-ANHYDRO-B-GLUCOSE",C6 H10 O5,Full match,No results,Full match,Partial match,Not the top hit,-0.00020,NaN,NaN,0.684
...,...,...,...,...,...,...,...,...,...,...,...
90,Quercetin,C15 H10 O7,Full match,Full match,Full match,Full match,Full match,0.00006,88.9,9.4,0.372
167,"3-Hydroxy-3,5,5-trimethyl-4-(3-oxo-1-buten-1-y...",C19 H30 O8,Full match,Full match,No results,No match,Partial match,-0.00015,90.1,9.5,1.671
168,"3,4-Bis(3-Methoxyanilino)cyclobut-3-ene-1,2-dione",C18 H16 N2 O4,No match,Full match,No results,Partial match,Partial match,-0.00074,54.9,7.7,0.318
169,L(-)-Fucose,C6 H12 O5,No results,Full match,No results,Not the top hit,Partial match,-0.00020,73.3,8.7,1.260


In [128]:
not_found

['Must_Oechsle',
 'Wine_Tartatic_Acid',
 'Wine_Malic_Acid',
 'Wine_Glucose',
 'Wine_Fructose',
 'Wine_Lactic_Acid',
 'Wien_Glycerol',
 'Wine_Acetic_Acid',
 'Altitude',
 'Average_slope',
 'Average_radiation',
 'Hydromorphie_code',
 'Exposition_South-West',
 'Geology_Gravel_moraine',
 'Geology_Molasse',
 'cv_rh',
 'median_rh',
 'cv_temperature',
 'median_temperature',
 'Dothideomycetes',
 'Hyaloscypha',
 'Cladosporium_austrohemisphaericum',
 'Alternaria_subcucurbitae',
 'Ascomycota',
 'Alternaria_eureka',
 'Hanseniaspora',
 'Mycosphaerella_tassiana',
 'Epicoccum_nigrum',
 'Basidiomycota',
 'Ustilago_hordei',
 'Alternaria_chlamydospora',
 'Sporobolomyces_roseus',
 'Naganishia_albida',
 'Cystofilobasidium_macerans',
 'Sordariomycetes',
 'Cordyceps_bassiana',
 'Stemphylium_solani',
 'Ascomycota.1',
 'Aureobasidium_pullulans',
 'Leucosporidiaceae',
 'Dothideales',
 'Cladosporium_cladosporioides',
 'Filobasidium_magnum',
 'Saccharomyces_cerevisiae',
 'Fungi',
 'Filobasidium_chernovii',
 'Doth

### Correlated metabolites

In [132]:
names_of_interest = [
    "3-Hexen-1-ol, (Z)-",
    "3-phenyl-5-(1,2,3-thiadiazol-4-yl)-1,2,4-oxadiazole",
    "1-Hexanol, 3,5,5-trimethyl-",
    "4-Acetamidobutanoic acid",
    "5-Hydroxytryptophol",
    "Acetobacter pasteurianus",
    "Azelaic acid",
    "Benzaldehyde, 2,4-dimethyl-",
    "Diethyl succinate",
    "Digalacturonic acid",
    "Farnesol, (2Z,6Z)-",
    "Galangin",
    "Hanseniaspora sp.",
    "NP-001596",
    "Hexadecanoic acid, ethyl ester",
    "Hexanal",
    "Proline-Hydroxyproline",
    "Mycosphaerella tassiana",
    "N-(5-acetamidopentyl)acetamide",
    "N-Methyldioctylamine",
    "Saccharomyces cerevisiae",
    "Tetradecane",
    "Tetrahydro lavandulyl acetate",
    "2,2-difluoro-N-[2-methyl-6-(trifluoromethyl)pyridin-3-yl]-2-(phenylthio)acetamide",
    "trans-Aconitic acid",
    "UDP-N-acetylglucosamine",
    "Cer-NS d34:3",
    "Norepinephrine",
    "Phe-Tyr",
    "L-Saccharopine",
    "5-Hydroxytryptophol",
    '3-(tert-butyl)-N-[2-hydroxy-3-(4-methoxyphenoxy)propyl]-1-methyl-1H-pyrazole-5-carboxamide',
    'Pro-Phe',
    '1-Palmitoyl-sn-glycero-3-phosphocholine',
    'L-Glutamic acid',
    'PEG n10', 
    'PEG n11',
    'Palmitoyl ethanolamide', 
    '2-Amino-1,3,4-octadecanetriol',
    'NP-020521' 
]

# Subset the dataframe
subset_df = result_df[result_df['Name'].isin(names_of_interest)]

# Check which names were not found
not_found = [name for name in names_of_interest if name not in result_df['Name'].unique()]
print("Not found in result_df:", not_found)


Not found in result_df: ['3-Hexen-1-ol, (Z)-', '1-Hexanol, 3,5,5-trimethyl-', 'Acetobacter pasteurianus', 'Benzaldehyde, 2,4-dimethyl-', 'Diethyl succinate', 'Farnesol, (2Z,6Z)-', 'Hanseniaspora sp.', 'Hexadecanoic acid, ethyl ester', 'Hexanal', 'Mycosphaerella tassiana', 'Saccharomyces cerevisiae', 'Tetradecane', 'Tetrahydro lavandulyl acetate', 'PEG n11']


In [136]:
# quick check if we're missing any LCMS metabolites

query = "PEG n11"
matches = find_columns_for_entry(query, entry_to_column)

if matches:
    for matched_entry, column_name in matches:
        print(f"Found: '{matched_entry}' corresponds to column '{column_name}'")
else:
    print("No matches found.")

No matches found.


In [134]:
subset_df.to_csv('LCMS-MS2-DIABLOmetabolites-correlated-annotation.csv')
subset_df

,Name,Formula,Annot. Source: Predicted Compositions,Annot. Source: mzCloud Search,Annot. Source: mzVault Search,Annot. Source: ChemSpider Search,Annot. Source: MassList Search,Annot. DeltaMass [Da],mzCloud Best Match,mzCloud Best Match Confidence,Ratio: (PostMLF) / (Blank)
6,1-Palmitoyl-sn-glycero-3-phosphocholine,C24 H50 N O7 P,Not the top hit,No results,Full match,Partial match,Full match,-0.00062,NaN,NaN,0.301
9,"2,2-difluoro-N-[2-methyl-6-(trifluoromethyl)py...",C15 H11 F5 N2 O S,No match,Full match,No results,No results,No results,0.00081,60.4,8.0,0.823
13,"2-Amino-1,3,4-octadecanetriol",C18 H39 N O3,Full match,Full match,Full match,Full match,Full match,-0.00047,79.8,9.0,1.361
17,3-(tert-butyl)-N-[2-hydroxy-3-(4-methoxyphenox...,C19 H27 N3 O4,Not the top hit,Full match,No results,Partial match,No results,-0.00045,60.0,46.2,2.756
18,"3-phenyl-5-(1,2,3-thiadiazol-4-yl)-1,2,4-oxadi...",C10 H6 N4 O S,Unused,Full match,No results,No results,No results,0.00112,62.5,8.1,1.519
19,4-Acetamidobutanoic acid,C6 H11 N O3,Full match,Full match,No results,Full match,Not the top hit,-0.00021,78.8,8.9,2.795
19,4-Acetamidobutanoic acid,C6 H11 N O3,Full match,Full match,No results,Full match,Not the top hit,-0.00021,78.8,8.9,2.795
24,5-Hydroxytryptophol,C10 H11 N O2,Full match,No results,Full match,Partial match,Partial match,-0.00026,NaN,NaN,1.836
24,5-Hydroxytryptophol,C10 H11 N O2,Full match,No results,Full match,Partial match,Partial match,-0.00026,NaN,NaN,1.836
39,Cer-NS d34:3,C34 H63 N O3,Full match,No results,Full match,No results,Partial match,-0.00076,NaN,NaN,1.731
